In [10]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import glob
import numpy as np
import time

### Merge same user's csv

In [4]:
users = ['weijen','yenho']

for user in users:
    same_user_files = glob.glob('./ppg2/*' + user + '.csv')
    merged_df = pd.DataFrame()

    for file in same_user_files:
        df = pd.read_csv(file)
        merged_df = pd.concat([merged_df, df], ignore_index=True)

    merged_df.sort_values('Time', inplace=True)
    merged_df.to_csv(user + '.csv', index=False)


### Data preprocessing

In [111]:
# 資料夾路徑
path ='./yenho.csv'
data = pd.read_csv(path)

# drop用不到的column
data = data.drop([' Green Count',
                  ' Green2 Count',
                  ' IR Count',
                  ' Red Count',
                  ' X Axis Acceleration (g)',
                  ' Y Axis Acceleration (g)',
                  ' Z Axis Acceleration (g)',
                  ' Sample Count',
                  ' Operating Mode',
                  ' SpO2 State',
                  ' SpO2 Percent Complete',
                  ' SCD State',
                  ' SAMPLE Time',
                  ' Walk Steps',
                  ' Run Steps',
                  ' KCal',
                  ' Tot. Act. Energy',
                  ' Ibi Offset'], axis=1)


# drop不合理的data
## 要刪的條件
condition = (data[' HR Confidence (%)'] < 10) | (data[' SpO2 Confidence (%)'] <= 0) | (data[' Unreliable R'] == 1) | (data[' Low Signal Quality'] == 1) | (data[' WSPO2 Low Pi'] == 1) | (data[' Unreliable R'] == 1)

## 删除滿足條件的data
data = data.drop(data[condition].index)

#drop用不到的
data = data.drop([' HR Confidence (%)',
                  ' RR Confidence (%)',
                  ' SpO2 Confidence (%)',
                  ' Low Signal Quality',
                  ' Unreliable R',
                  ' WSPO2 Low Pi'], axis=1)

### Real Time心情

In [4]:
def calculate_mood_score(data):
    mood_score = 0
    
    # motion flag
    if data[' Motion Flag'] == 0:
        mood_score += 5

    # HR
    if data[' Heart Rate (bpm)'] < 60 or data[' Heart Rate (bpm)'] > 100:
        mood_score -= 3
    else:
        mood_score += 2

    # SpO2
    if data[' SpO2 (%)'] >= 95:
        mood_score += 3
    else:
        mood_score -= 2

    print("心情指數:", mood_score)

for i in data.index:
    calculate_mood_score(data.loc[i])


心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心情指數: 10
心

### 健康狀況預測(前10分鐘的資料預測下一分鐘的平均)

In [112]:
data = data.drop(['Time',' Activity',' Motion Flag'],axis = 1)
condition = (data[' RR'] == 0)
data = data.drop(data[condition].index)

In [113]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential



# 创建时间窗口
def create_time_windows(data, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size-60):
        window = data.iloc[i:i+window_size,:]
        target = data.iloc[i+window_size:i+window_size+60,:]
        X.append(window)
        y.append(target)
        # print(y)
    return np.array(X), np.array(y)

window_size = 600 
X , y = create_time_windows(data, window_size)
y = y.reshape(y.shape[0], -1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [117]:
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 创建和训练LSTM模型
model = Sequential()
model.add(LSTM(512, input_shape=(window_size, 4), return_sequences=True))
model.add(LSTM(256))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(32))
model.add(Dense(240))  # 修改为 (60 * 4 = 240) 的输出形状
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=15, batch_size=32)

# 进行预测
predictions = model.predict(X_test)
predictions = predictions.reshape((-1, 60, 4))  # 将预测结果重新调整为 (60, 4) 的形状

# 打印预测结果
print(predictions)


Epoch 1/15
14/14 [==============================] - 20s 1s/step - loss: 115042.7812
Epoch 2/15
14/14 [==============================] - 19s 1s/step - loss: 100265.6484
Epoch 3/15
14/14 [==============================] - 19s 1s/step - loss: 51195.1914
Epoch 4/15
14/14 [==============================] - 19s 1s/step - loss: 12165.8008
Epoch 5/15
14/14 [==============================] - 19s 1s/step - loss: 4806.8706
Epoch 6/15
14/14 [==============================] - 18s 1s/step - loss: 3612.5474
Epoch 7/15
14/14 [==============================] - 19s 1s/step - loss: 3388.8889
Epoch 8/15
14/14 [==============================] - 18s 1s/step - loss: 3319.4358
Epoch 9/15
14/14 [==============================] - 18s 1s/step - loss: 3313.3601
Epoch 10/15
14/14 [==============================] - 18s 1s/step - loss: 3309.0906
Epoch 11/15
14/14 [==============================] - 19s 1s/step - loss: 3307.0039
Epoch 12/15
14/14 [==============================] - 19s 1s/step - loss: 3303.1687
Epoch 1

### 久坐提醒

In [ ]:
# 設定休息閾值（以秒為單位）
rest_threshold = 60  
# 開始記錄時間
start_time = time.time()

while True:
    # 獲取當前狀態（假設從某個地方獲取，例如健康手錶）
    current_status = get_current_status()  # 替換為獲取當前狀態的函式

    if current_status == "REST":
        # 如果是休息狀態，檢查是否休息太久
        elapsed_time = time.time() - start_time
        if elapsed_time >= rest_threshold:
            print("您已經休息太久了！請起身活動一下。")
            # 在這裡可以添加提醒的相關邏輯或程式碼

    elif current_status == "WALK":
        # 如果是行走狀態，重置計時器
        start_time = time.time()

    # 可以根據需要調整檢查狀態的間隔
    time.sleep(10)  # 每10秒檢查一次狀態


### 沒戴好通知